In [1]:
from __future__ import annotations
from typing import Iterable, Literal
from typing_extensions import Self


def parser(file = 'input.txt'):
    with open(file, 'r') as file:
        # for line in file:
        #     yield line.strip('\n')
        workflows_raw, cases = file.read().split('\n\n')
        workflows = {}
        for wf in workflows_raw.split('\n'):
            key = wf[:wf.index('{')]
            rules_raw = wf[wf.index('{')+1:-1]
            rules = [(rule[0], rule[1], int(rule[2:rule.index(':')]), rule[rule.index(':')+1:]) if ':' in rule else rule for rule in rules_raw.split(',')]
            possible_destinations = [r[-1] for r in rules[:-1]] + [rules[-1]]
            # print(key, rules, possible_destinations)
            if key in workflows:
                raise Exception()
            workflows[key] = (rules, possible_destinations)
        cases = [{combo[0]:int(combo[2:]) for combo in case[1:-1].split(',')} for case in cases.split('\n')]
        # print(cases)
        return workflows, cases


def solver(workflows, cases):
    # print([key for key, (rules, dests) in workflows.items() if 'A' in dests])
    summed = 0
    for case in cases:
        # print("\nCASE")
        curr = 'in'
        while not curr or curr not in 'AR':
            # print(curr)
            rules, _ = workflows[curr]
            else_dest = rules[-1]
            
            for rule in rules[:-1]:
                var, op, value, new = rule
                if {'>': int.__gt__, '<': int.__lt__}[op](case[var], value):
                    curr = new
                    break
            else:
                curr = else_dest
        if curr == 'A':
            # print(case, curr)
            summed += sum(case.values())
    return summed


if __name__ == '__main__':
    test_p1 = solver(*parser('test_input.txt'))
    print(test_p1)
    assert test_p1 == 19114, test_p1
    print(solver(*parser()))

    # test_p2 = solver(parser('test_input.txt'))
    # test_p2 = solver(parser('test_input_p2.txt'))
    # print(test_p2)
    # assert test_p2 == None, test_p2
    # print(solver(parser()))

19114
472630


In [2]:
S = int | Literal['-inf']
E = int | Literal['inf']

class infrange:
    start: S
    end: E

    def __init__(self, start: S, end: E):
        self.start = start
        self.end = end
    
    def __contains__(self, n: S | E | float):
        if n == 'inf' or n == '-inf':
            return float(n) in self

        match (self.start, self.end):
            case '-inf', 'inf':
                return True
            case s, 'inf' if isinstance(s, (int, float)):
                return s <= n
            case '-inf', e if isinstance(e, (int, float)):
                return e > n
            case s, e if isinstance(s, (int, float)) and isinstance(e, (int, float)):
                return s <= n < e
        raise Exception
    
    def __and__(self, other: Self):
        if not (self.end in other or self.start in other or other.start in self or other.end in self):
            return infrange(0, 0)

        match (self.start, other.start):
            case '-inf', '-inf':
                start = '-inf'
            case _, '-inf':
                start = self.start
            case '-inf', _:
                start = other.start
            case a, b:
                start = max(a, b)
        
        match (self.end, other.end):
            case 'inf', 'inf':
                end = 'inf'
            case _, 'inf':
                end = self.end
            case 'inf', _:
                end = other.end
            case a, b:
                end = min(a, b)
        
        return infrange(start, end)
    
    # def __or__(self, other: Self):
    #     match (self.start, other.start):
    #         case 'inf', 'inf':
    #             start = 'inf'
    #         case _, 'inf':
    #             start = 'inf'
    #         case 'inf', _:
    #             start = 'inf'
    #         case a, b:
    #             start = min(a, b)
        
    #     match (self.end, other.end):
    #         case 'inf', 'inf':
    #             end = 'inf'
    #         case _, 'inf':
    #             end = 'inf'
    #         case 'inf', _:
    #             end = 'inf'
    #         case a, b:
    #             end = max(a, b)
        
        return infrange(start, end)
    
    def __repr__(self):
        return f'infrange(start={self.start}, end={self.end})'
    
    def __bool__(self):
        match (self.start, self.end):
            case '-inf', 'inf':
                return True
            case '-inf', _:
                return True
            case _, 'inf':
                return True
            case s, e if isinstance(s, int) and isinstance(e, int):
                return s < e
        raise Exception
    
    def __len__(self):
        match (self.start, self.end):
            case '-inf', 'inf':
                return float('inf')
            case '-inf', _:
                return float('inf')
            case _, 'inf':
                return float('inf')
            case a,b:
                return b - a
    
    def __iter__(self):
        if self.start == '-inf' or self.end == 'inf':
            raise Exception
        return range(self.start, self.end).__iter__()

print(infrange('-inf', 5) & infrange(3, 'inf'))
# print(infrange(2, 8) | infrange('inf', 5))
print(infrange(0, 'inf') & infrange(10, 'inf'))
print(infrange('-inf', 5) & infrange(10, 'inf'))
print(infrange(10, 'inf') & infrange('-inf', 5))

infrange(start=3, end=5)
infrange(start=10, end=inf)
infrange(start=0, end=0)
infrange(start=0, end=0)


In [8]:
from collections import defaultdict
from pprint import pprint

def print2(*args, **kwargs):
    ...


def solver(workflows, cases):
    print2([key for key, (rules, dests) in workflows.items() if 'A' in dests])
    queue = [['A', {c:infrange(0, 'inf') for c in 'xmas'}]]
    summed = 0
    case = {c:infrange(0, 'inf') for c in 'xmas'}

    def r(curr: str, ranges: dict[str, list[infrange]]):
        # 1 ranges dict in place bij houden
        # over de rest heen loopen, bij elke optie splitten
        if curr == 'A':
            print2('WAS A ', ranges)
            return [ranges]
        elif curr == 'R':
            return []

        options = []
        print2("CURR: ", curr)
        (*rules, else_dest), _ = workflows[curr]
        for var, op, value, new in rules:
            print2("VAR ", var, op, value)
            if op == '<':
                ir = [ar for r in ranges[var] if (ar:=infrange(0, value) & r)]
                nr = [ar for r in ranges[var] if (ar:=infrange(value, 'inf') & r)]
            elif op == '>':
                ir = [ar for r in ranges[var] if (ar:=infrange(value + 1, 'inf') & r)]
                nr = [ar for r in ranges[var] if (ar:=infrange(0, value + 1) & r)]
            else:
                raise Exception
            print2('ranges', ir, nr)
            if ir:
                c = dict(ranges)
                c[var] = ir
                print2('calling new: ', new, c)
                t = r(new, c)
                print2('return of `correct` ', t)
                options.extend(t)
            if nr:
                ranges[var] = nr
            else:
                raise Exception()
        print2("CALLING ELSE ", else_dest, ranges)
        t = r(else_dest, dict(ranges))
        print2('return of else ', t)
        options.extend(t)
        print2()
        print2()
        return options
        

    ret = r('in', {c:[infrange(1, 4000+1)] for c in 'xmas'})
    # pprint(ret)

    # counter = defaultdict(int)
    # for c in 'xmas':
    #     s = set()
    #     for ranges in ret:
    #         for subrange in ranges[c]:
    #             s |= {*subrange}
    #     counter[c] += len(s)
    # print(counter)
    n = 0
    for sol in ret:
        temp = 1
        for ls in sol.values():
            if len(ls) != 1:
                raise Exception(ls)
            temp *= len(ls[0])
        n += temp

    

    print(n)
    return n
    return summed


if __name__ == '__main__':
    test_p2 = solver(*parser('test_input.txt'))
    # test_p2 = solver(parser('test_input_p2.txt'))
    print(test_p2)
    # assert test_p2 == None, test_p2
    print(solver(*parser()))
    self_input = solver(*parser('self_input.txt'))
    print(self_input)

167409079868000
167409079868000
116738260946855
116738260946855
162708000000000
162708000000000
